In [1]:
import pandas as pd
import numpy as np
url = 'https://github.com/mattharrison/datasets/raw/master/data/vehicles.csv.zip'
df = pd.read_csv(url)

/var/folders/61/0_k_4p5x6ps6rb74cyv2c5c80000gn/T/ipykernel_42530/1524119719.py:4: DtypeWarning: Columns (68,70,71,72,73,74,76,79) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


In [2]:
city_mpg = df.city08
highway_mpg = df.highway08

city_mpg, highway_mpg

(0        19
 1         9
 2        23
 3        10
 4        17
          ..
 41139    19
 41140    20
 41141    18
 41142    18
 41143    16
 Name: city08, Length: 41144, dtype: int64,
 0        25
 1        14
 2        33
 3        12
 4        23
          ..
 41139    26
 41140    28
 41141    24
 41142    24
 41143    21
 Name: highway08, Length: 41144, dtype: int64)

<h2>Manipulation Methods</h2>

Manipulation methods help you understand and cleanup the data. They operate on a series and return a new series. These methods usually preserve the index.

<h5>.apply and .where</h5>

The .apply method allows you to apply a function element-wise to every value. 
<p>If you pass in a NumPy function that works on an array, it will broadcast the operation to the series.</p>

It is generally not good to use because it typically operates on every value, so if your data has a million values, it will be called 1 million times.

This breaks the fast, vectorized patterns and falls back to using loop style code.

The .where method is optimized, takes a bollean array, and can include an 'other' value for the values that evaluate False.

The .mask does the opposite of .where. It will keep the values that evaluate false and replace the True values.

In [3]:
def gt20(val):
    return val > 20

In [5]:
%%timeit
city_mpg.apply(gt20)

32.1 ms ± 5.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%%timeit
city_mpg.gt(20)

589 µs ± 58.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
make = df.make
top_5 = make.value_counts().index[:5]
top_10 = make.value_counts().index[:10]

In [8]:
def generalize_top5(val):
    if val in top_5:
        return val
    return 'Other'

In [9]:
make.apply(generalize_top5)

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

In [11]:
make.where(make.isin(top_5), 'Other')

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

In [13]:
make.mask(~make.isin(top_5), 'Other')

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

In [12]:
make.isin(top_5)

0        False
1        False
2         True
3         True
4        False
         ...  
41139    False
41140    False
41141    False
41142    False
41143    False
Name: make, Length: 41144, dtype: bool

<h2>np.select</h2>

The Numpy function select works with pandas series. It takes a list of bool arrays and a list with corresponding replacement values. You can also pass it a default value for values that don't return true in any of the boolean arrays.

In [14]:
np.select([make.isin(top_5), make.isin(top_10)], [make, 'Top 10'], 'Other')

array(['Other', 'Other', 'Dodge', ..., 'Other', 'Other', 'Other'],
      dtype=object)

<h2>Missing Data</h2>

It is important to handle missing data because many machine learning algorithms won't work with missing data.
It's also good to know how much data you are missing.

The .isna() method of a series will return a bool array, where the value is true if that source value is missing.

In [15]:
cyl = df.cylinders
cyl.isna().sum()

206

If you index .loc with a boolean array, it returns the rows where the boolean array is true.

In [16]:
missing = cyl.isna()
make.loc[missing]

7138     Nissan
7139     Toyota
8143     Toyota
8144       Ford
8146       Ford
          ...  
34563     Tesla
34564     Tesla
34565     Tesla
34566     Tesla
34567     Tesla
Name: make, Length: 206, dtype: object

Use .fillna to replace the na values...

In [17]:
cyl.fillna(0).loc[7136:7141]

7136    6.0
7137    6.0
7138    0.0
7139    0.0
7140    6.0
7141    6.0
Name: cylinders, dtype: float64

<h2>Interpolate</h2>

You can use .interpolate to fill in missing values in ordered data. It interpolates the previous and next values.

In [19]:
temp = pd.Series([32,40,None, 42, 39, 32])
fixed = temp.interpolate()
fixed

0    32.0
1    40.0
2    41.0
3    42.0
4    39.0
5    32.0
dtype: float64

<h2>Clipping Data</h2>

The .clip method lets you clip outliers to a specified range.

To trim the values to be between the 5th and 95th quintile...

In [20]:
city_mpg.loc[:446].clip(lower=city_mpg.quantile(.05), upper=city_mpg.quantile(.95))

0      19.0
1      11.0
2      23.0
3      11.0
4      17.0
       ... 
442    15.0
443    15.0
444    15.0
445    15.0
446    27.0
Name: city08, Length: 447, dtype: float64

<h2>Sorting Values</h2>

The .sort_values method will sorth the values in ascending order and rearrange the index accordingly.

In [21]:
city_mpg.sort_values()

7901       6
34557      6
37161      6
21060      6
35887      6
        ... 
34563    138
34564    140
32599    150
31256    150
33423    150
Name: city08, Length: 41144, dtype: int64

Because of index alignment, math and many other operations work on a sorted series...

In [22]:
(city_mpg.sort_values() + highway_mpg) / 2 

0        22.0
1        11.5
2        28.0
3        11.0
4        20.0
         ... 
41139    22.5
41140    24.0
41141    21.0
41142    21.0
41143    18.5
Length: 41144, dtype: float64

Sorting the index will sort the index in ascending order...

In [23]:
city_mpg.sort_values().sort_index()

0        19
1         9
2        23
3        10
4        17
         ..
41139    19
41140    20
41141    18
41142    18
41143    16
Name: city08, Length: 41144, dtype: int64

<h2>Dropping Duplicates</h2>

The .drop_duplicates method will remove values that appear more than once. 

The keep parameter determines what to keep; 'first', 'last', or False. False removes all duplicated values. The default is first.

In [24]:
city_mpg.drop_duplicates()

0         19
1          9
2         23
3         10
4         17
        ... 
34364    127
34409    114
34564    140
34565    115
34566    104
Name: city08, Length: 105, dtype: int64

<h2>Ranking Data</h2>

.rank will return a series that keeps the original index but uses the rankes of values from the original series.

In [25]:
city_mpg.rank()

0        27060.5
1          235.5
2        35830.0
3          607.5
4        19484.0
          ...   
41139    27060.5
41140    29719.5
41141    23528.0
41142    23528.0
41143    15479.0
Name: city08, Length: 41144, dtype: float64

Min puts equal values in the same rank...

In [27]:
city_mpg.rank(method='min')

0        25555.0
1          136.0
2        35119.0
3          336.0
4        17467.0
          ...   
41139    25555.0
41140    28567.0
41141    21502.0
41142    21502.0
41143    13492.0
Name: city08, Length: 41144, dtype: float64

Dense won't skip any positions...

In [28]:
city_mpg.rank(method='dense')

0        14.0
1         4.0
2        18.0
3         5.0
4        12.0
         ... 
41139    14.0
41140    15.0
41141    13.0
41142    13.0
41143    11.0
Name: city08, Length: 41144, dtype: float64

<h2>Replacing Data</h2>

The .replace method allows you to map values to new values.

You can replace values directly or use a dictionary to map values to their new values.


In [29]:
make.replace('Subaru', 'SUB')

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4               SUB
            ...    
41139           SUB
41140           SUB
41141           SUB
41142           SUB
41143           SUB
Name: make, Length: 41144, dtype: object